<a href="https://colab.research.google.com/github/antonFJohansson/Article_collection/blob/master/Understanding_predictive_information_criteria_for_Bayesian_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-----------------------------------------------
This article will review some information criteries and frame them in a Bayesian way. The considered information criterias are
* AIC
* DIC
* WAIC

And also some investigation into cross-validation. What we want is to estimate the expected out-of-sample error based on a bias-corrected version of the within-sample-error. [Q1]

All of these information criterias can be viewed as an approximation to some form of cross-validation [They do not show this in the article though]. There are some controversies with all these methods (mainly DIC) but that is because they attempt at obtaining an unbiased estimate of the true prediction error, given only the data used to fit the algorithm, and they have to be quick to compute so that people will actually use them for their methods. In this article they will focus on the metric given by predictive accuracy and this will determine which of the models to select.
Throughout this article we will work with a likelihood for our data as 
$$p(y|\theta,x) = \prod_{i=1}^np(y_i|\theta,x_i)$$
We will most of the time suppress the dependency on $x$ in our notations.

There are two kind of measures for predictive accuracy that we can be interested in, one for point estimates and one for probabilitistic predictions. Measures of predictive accuracies for point estimates are called scoring functions (e.g mean squared error), for probabilistic predictions they are called scoring rules (e.g logarithmic and zero-one scores).[Q2] 

A common summary of the predictive fit is the loglikelihood/log-predictive-density. The theory behind this rule is that in the limit of high-data we have the property that the model with the smallest KL to the true data generating distribution will also have the highest expected loglikelihood. [Q3] Thus we can use this as a criteria for how well the model fits the data [Q4].

## Questions/Answers
* [Q1]: I guess we say that it is bias corrected because the estimated error will have been shifted a bit from its true position due to using within-sample-error. And not bias in the sense that the model cannot fit the data?
* [Q2]: Why is zero-one score a probabilistic scoring rule? Is it because we try and predict a few different classes and we only get 0 loss when predicting the right class? In that case I guess also cross-entropy is a scoring rule.
* [Q3]: They mention here that this also gives that the model will have the highest posterior probability
* [Q4]: I do not understand how we would use this in a Bayesian setting though, we do not have a single parameter value $\theta$ that we can plug in right?

--------------------------------------------------
Under standard conditions the posterior density will approach a normal distribution in the limit of high amounts of data. In this limit the posterior is dominated by the likelihood since it contributes $n$ terms while the prior only contributes one term, the likelihood will therefore also be of Normal density shape. Denoting the asymptotic Normal distribution as $\theta|y \sim \mathcal{N}(\theta, V_0/n)$, then we get the expression for the loglikelihood as
$$\log p(y|\theta) = c(y) -\frac{1}{2}(k \log(2\pi)+\log|V_0/n| + (\theta-\theta_0)^T(V_0/n)^{-1}(\theta-\theta_0))$$
This follows since we have
$$\frac{p(y|\theta)}{p(y)} \approx \frac{p(y|\theta)p(\theta)}{p(y)} = p(\theta|y)$$
in the large data limit. From this we can see that since $\theta|y \sim \mathcal{N}$, the loglikelihood will have a density given by a $\chi_d^2$ density, where $d$ is the dimension of $\theta$. For singular models where a set of different parameters can map to the same model, this aysmptotic result above will not hold. One needs to analyze these models in a different way.

The ideal measure of a models predictive accuracy would be its performance on new unseen data, i.e, out-of-sample prediction performance for data generated from the same distribution $f$ that generated the training data. The out-of-sample predictive fit for a new datapoint is given as
$$\log p_post(\tilde{y}) = \log \mathbb{E}_{post}(p(\tilde{y}|\theta)) = \log \int p(\tilde{y}|\theta)p_{post}(\theta)d\theta$$
But since we do not know the new datapoint $\tilde{y}$ we must define a new quantity know as expected-(out-of-sample)-log-predictive-density (elpd) (also known as mean log predictive density) as
$$elpd = \mathbb{E}_f(\log p_{post}(\tilde{y})) =\int \log p_{post}(\tilde{y}) f(\tilde{y})d\tilde{y}$$
Now they also say that in order to keep comparability with the current data set we can define "expected log pointwise predictive density for a new dataset" (elpdd) which is given as [Q5]
$$elpdd = \sum_{i=1}^n\mathbb{E}_f(\log p_{post}(\tilde{y_i}))$$
And sometimes we can do all of the above given a point estimate for our parameter $\theta$, $\hat{\theta}$.


Now this might connect with some of the things I mentioned above but they write that in general the parameter $\theta$ is not known and thus we would like to work with the posterior distribution $p(\theta|y)$ instead. Then we could summarize the model fit with the log pointwise predictive accuracy as
$$lppd = \log \prod_{i=1}^n p_{post}(y_i) = \sum_{i}\log \int p(y_i|\theta)p_{post}(\theta)d\theta$$
Given posterior samples, we can easily evaluate this with Monte Carlo sampling as
$$lppd \approx \sum_{i=1}^n\bigg(\frac{1}{S}\sum_{j=1}^Sp(y_i|\theta_j)\bigg)$$
We will see that this term is an underestimate of the true lppd since we only use the observed data, i.e, the error on our training data will make our model appear better than it actually is. Thus we are interested in some form of bias correction for this term. There are still some questions regarding how to implement this, and it can depend on how our experiment is performed, is the number of new observations a random variable or fixed, are we working in a hierarchical model or not etc. To take these effects into account in a cross-validation scheme one can use something called h-block cross-validation. [Variations similar to this but for Information Criterias have not been proposed].






## Questions/Answers:
* [Q5]: I do not see how we can get different things in each term in this sum, it should all be the same in my opinion?
* [A5]: I think there could be a difference to consider $n$ datapoints at one time, given that we do not assume independence between them.

----------------------------------------------------------
Usually Information Criterias are based on a measure of the Deviance which is the log predictive density given a point prediction for the parameters multiplied with -2, i.e, $-2\log p(y|\hat{\theta})$ If two models are equally complex then one can simply compare their log predictive densities straight away to get an idea of which model is the best (is this true though? I guess they do not deal with singular models or model with multiple minimas maybe), but when the models are of different sizes it is important to somehow penalize the complexity since a higher capacity model always is able to fit the training data better. Since we do not have the true data generating distribution we have to approximate the log predictive density in some way. We can do that as
* Use the observed training data.
* Adjust for bias, such as in AIC where we subtract a term based on the bias. These methods are **correct only in expectations** and thus we cannot say anything in a single case when we compare two different methods [Q6].
* Corss-validation. CV is also tied to the data at hand and can thus only at best be correct in expectations. Is this since we cannot know which data we actually obtained and the data we obtained can prefer other methods?

**AIC** For AIC we work with point estimates of the parameters, we thus try and estimate the quantity of the expected log predictive density $\log \mathbb{E}_fp(\tilde{y}|\hat{\theta}(y))$, where both $\tilde{y}$ and $y$ are random. This is not possible to estimate since we do not know $f$ and thus we instead work with the observed data and try and correct for the bias. AIC relies on asymptotic normality of the likelihood and the by subtracting the number of parameters $k$, we subtract an estimate for how much $k$ parameters would increase the predictive accuracy by chance alone. Thus we obtain the formula as
$$\hat{elpd}_{AIC} = \log p(y|\hat{\theta}_{MLE}) - k$$
AIC is the above multiplied with -2. This formula does not work easily beyond linear models with flat priors, this is since how to measure the number of parameters is not completely clear (there are extensions which deal with effective number of parameters but these are apparently difficult to work with).


**DIC** DIC works in a similar way to AIC but instead of the MLE estimate, we take the posterior mean estimate of $\theta$. And instead of a penalty based on the number of parameters, DIC involves a penalty based on the data, thus the new term is given as
$$\hat{elpd}_{DIC} = \log p(y|\hat{\theta}_{postmean}) - p_{DIC}$$
where $p_{DIC}$ is a measure of the number of effective amount of parameters and is defined as
$$p_{DIC} = 2\bigg(\log p(y|\theta_{postmean}) -\mathbb{E}_{post}(\log p(y|\theta))\bigg)$$
Thus we can easily sample the last expectation to get an estimate of the effective number of parameters. They write [The posterior mean of $\theta$ will produce the maximum log predictive density when it happens to be same as the mode, but I do not really see this. Should it not be maximum if $\theta$ is given by the MLE estimate, per definition. Unless they mean the whole expression elpd.] There is a slightly adjusted formula as well given by
$$p_{DICalt} = 2var_{post}(\log(p(y|\theta)))$$.
The advantage with the alternative measure is that it is always positive, the original $p_{DIC}$ can be negative. The first one is more numerically stable though. For linear models with a uniform prior distribution, both of these measures reduce to the number of parameters $k$ [Q7]. The reason this measure was made popular was due to easy evalutation through MCMC samples. The actual DIC is measured with deviance as before to obtain, 
$$DIC = -2\log p(y|\hat{\theta}_{Bayes}) + 2p_{DIC}$$

## Questions/Answers
* [Q6]: But I guess there are possibilities to sample new data or something similar and see which model is preferred in expectations with these methods then?
* [Q7]: They say that these measures reduce to $k$ for linear models but this is not completely clear to me.

-------------------------------------------------------------------------------------
There is also the criteria known as WAIC (Widely Applicable Information Criteria). This is a fully Bayesian approach to estimate the out-of-sample expectation. It has a similar structure to the earlier ones, a datafit term (log pointwise posterior predictive) and a penalty term for the amount of parameters. There are two ways to calculate these penalty terms, the first is given as
$$p_{WAIC1} = 2\sum_{i=1}^n\bigg(\log (\mathbb{E}_{post} p(y_i|\theta)) - \mathbb{E}_{post}(\log p(y_i|\theta))\bigg)$$
Which is easily calculated by simulations as before, The other correction term is given as
$$p_{WAIC2} = \sum_{i=1}^nvar_{post}(\log p(y_i|\theta))$$
Which can be estimated with the sample variance. We then can use either of these terms as the bias correction and obtain
$$\hat{elppd}_{WAIC} = lppd - p_{WAIC}$$
Once again if we are working with a linear model one can show that this is approximately the same as a penalty on the number of parameters. There are also some connections between this one and cross-validation. One advantage of WAIC is that we do not rely on any point estimate when working with this formula, we use the full posterior distribution. WAIC also works with singular models.

A difference between WAIC and DIC compared to AIC is that these have penalty terms that depend on the observed data. This can make sense in certain situations such as for example, when working with a model $y_1,...y_n \sim \mathcal{N}(\theta,1)$ and a prior $p(\theta) = U(0,\infty)$. If we observe all $y$ close to 0, then there is almost nothing to estimate since the parameter $\theta$ must be close to 0, thus the effective number of parameter will be less than 1 (I imagine the extreme case when we observe only 0 values, then $\theta$ must be 0 in some sense and we cannot estimate it). While on the other hand if we observe high values for all $y$, then there is still uncertainty left in the parameter $\theta$ and the effective number of parameters is 1 [Q8].

There is also the BIC measure but they say that the goal of BIC is to simply estimate the marginal distribution of the data and does not rely on estimating the out-of-sample accuracy. They do not consider it further.



## Questions/Answers
* [Q8]: They mentioned online that effective number of parameters can be easily seen if we are working in regularisation, such as L1 where even if we have many parameters, some of them has to be forced to 0 and we actually do not have that many parameters. Can we see this similarly here but instead with the uncertainty sort of, how much the uncertainty is reduced about one parameter given that we know another?

----------------------------------------------------------------------
There is also the possibility of working with Bayesian Cross-Validation where we split the data into $y_{train}$ and $y_{holdout}$, we train our model on $y_{train}$, obtain a posterior distribution and use that distribution to predict the values on the hold out set as $\log p_{train}(y_{holdout}) = \log \int p(y_{holdout}|\theta)p(\theta|y_{train})$. This can the be approximated with Monte Carlo sampling.
We focus here on the LOO-CV in which we have the term given as 
$$lppd = \sum_{i=1}^n \log p_{post(-i)}(y_i)$$
which we can calculate with Monte Carlo sampling as before. Each prediction is conditioned on $n-1$ datapoints which will thus underestimate the predictive fit (since we use less data than what we have available). For large $n$ this is not an issue but for small $n$ it can cause some issues. There are ways to correct for this bias but it is rarely used. There are some shortcuts to avoid having fit $n$ models here but we will not deal with them in this article.

There are also some relations between cross-validation and these information criterias, such as
* AIC has been shown to be asymptotically equivalent to LOO-CV computed using the maximum likelihood estimate.
* DIC is a variation of the **regularized information criteria** which has been shown to be asymptotically equivalent to LOO-CV using plug-in predictive densities
* Bayesian CV also works with singular models and is asymptotically equivalent with WAIC, for small $n$ there can be a significant difference.


--------------------------------------------------------------------------------------
In this part we will evaulate these criterias in some simple situations. First we consider Normal data with a uniform prior distribution as $y_1,...,y_n \sim \mathcal{N}(\theta,1)$ with a noninformative prior distribution given as $p(\theta) \propto 1$.
* AIC: The formula is given as
$$\hat{elpd}_{AIC} = \log p(y|\hat{\theta}_{MLE}) - k$$
The MLE estimate is given by $\theta = \bar{y}$ and the log predictive density is given by
$$\log p(y|\hat{\theta}_{MLE}) = -\frac{n}{2}\log(2\pi) - \frac{1}{2}\sum_{i=1}^n(y_i-\bar{y})^2 = -\frac{n}{2}\log(2\pi) - \frac{1}{2}(n-1)s_y^2$$
Thus the AIC value is given by
$$\hat{elpd}_{AIC} = -\frac{n}{2}\log(2\pi) - \frac{1}{2}(n-1)s_y^2 - 1$$
[**It is interesting that the prior term actually has no effect at all here.**]
* DIC: The formula is given as
$$\hat{elpd}_{DIC} = \log p(y|\hat{\theta}_{postmean}) - p_{DIC}$$
where 
$$p_{DIC} = 2\bigg(\log p(y|\theta_{postmean}) -\mathbb{E}_{post}(\log p(y|\theta))\bigg)$$
Thus we need the posterior mean and the posterior distribution to estimate the expectations. We can derive the posterior density to be
$$p(\theta|y) = \mathcal{N}(\theta;\bar{y},\sqrt{1/n}^2)$$
Calculating the term given as (the first term is identical with the first AIC term)
$$\mathbb{E}_{post}(\log p(y|\theta)) = -\frac{1}{2}\log(2\pi)-\frac{1}{2}[(n-1)s_y^2+1]$$. Thus the total term for DIC is
$$\hat{elpd}_{DIC} = -\log p(y|\hat{\theta}_{postmean}) + \mathbb{E}_{post}(\log p(y|\theta)) =  -\frac{n}{2}\log(2\pi) - \frac{1}{2}(n-1)s_y^2 - 1$$
So AIC = BIC in this case (but mostly because we didn't add any prior information I presume).
* WAIC: The formula for WAIC is given as
$$\hat{elpd}_{WAIC} = \sum_{i=1}^n \log p_{post}(y_i) - p_{WAIC}$$
So first we need the posterior predictive distribution. This distribution can be obtained to be $p_{post}(y) = \mathcal{N}(y|\bar{y},1+\frac{1}{n})$, thus summing these terms for all data points we obtain
$$\sum_{i=1}^n -\frac{n}{2}\log(2\pi)-\frac{n}{2}\log(1+\frac{1}{n}) - \frac{1}{2}\frac{n(n-1)}{n+1}s_y^2$$
Next we will determine the two forms of effective number of parameters which are given as,
$$p_{WAIC1} = 2\sum_{i=1}^n\bigg(\log (\mathbb{E}_{post} p(y_i|\theta)) - \mathbb{E}_{post}(\log p(y_i|\theta))\bigg)$$
and
$$p_{WAIC2} = \sum_{i=1}^nvar_{post}(\log p(y_i|\theta))$$
For the first term we obtain
$$2\sum_{i=1}^n\bigg(\log p(y_i|y) + \frac{1}{2}(\log(2\pi)+(y_i-\bar{y})^2+\frac{1}{n})\bigg) = 2\sum_{i=1}^n\log p(y_i|y) + 2\frac{n}{2}\log(2\pi)+(n-1)s_y^2+1$$
Combining this gives
$$p_{WAIC1} = \frac{n-1}{n+1}s_y^2+1-n\log(1+\frac{1}{n})$$
And similarly we can get $p_{WAIC2}$ as
$$p_{WAIC2} = \frac{n-1}{n}s_y^2+\frac{1}{2n}$$
So we can see that there are differences between WAIC and AIC and DIC. The effective number of parameters for WAIC is less than 1, in the limit of large $n$ we can replace $s_y^2$ by it's expected value of $1$ (we have assumed the data generating distribution to be the normal model and $s_y^2$ is an unbiased estimator of the variance) which gives $p_{WAIC} \rightarrow 1$.
* Cross-validation: For LOO-CV we want the posterior predictive $p_{post-i}(y_i) = \mathcal{N}(y_i;\bar{y}_{-i},1+1/(n+1))$, thus we get the sum of these terms as
$$\sum_{i=1}^n \log p_{post-i}(y_i) = \frac{n}{2}\log (2\pi)-\frac{n}{2}\log(1+\frac{1}{n-1}) - \frac{1}{2}\frac{n-1}{n}\sum_{i=1}^n(y_{-i}-\bar{y}_{-i})^2$$
It is also possible to add a bias corrected term to this expression [Q9]

## Questions/Answers
* [Q9]: This bias correction was for the fact that we estimate these terms using only $n-1$ data points and $n$ data points right? 


-------------------------------------------------------
We now want to compare these terms with the expectation of these quantities over new data. Thus since we know the generative distribution we are interested in evaluating
$$elppd = \sum_{i=1}^n\mathbb{E}(\log p_{post}(\tilde{y}_i)) = -\frac{n}{2}\log(2\pi)-\frac{n}{2}\log(1+\frac{1}{n})-\frac{1}{2}\frac{n}{n+1}\sum_{i=1}^n\mathbb{E}\big((\tilde{y}_i-\bar{y})^2\big)$$
Thus we are interested in the expected value of the quantities that we try and evaluate. The last term we can evaluate as (expectation is taken w.r.t both the new datapoints $\tilde{y}$, but also those already observed $\bar{y}$,so we calculate the expected value when we repeat the entire experiment and calculate on new points.)
$$\mathbb{E}\bigg(\sum_{i=1}^n(\tilde{y}_i-\bar{y})^2 \bigg) = (n-1)\mathbb{E}(s_y^2) +n\mathbb{E}((\bar{\tilde{y}}-\bar{y})^2)$$
We rewrite the sum such that the $\tilde{y}$ are subtracted towards the mean of those points, and the second term is just the difference of the means. Now we have assumed these points are obtained iid and both the new and old points have the same distribution and thus we get
$$\bar{y} \sim \mathcal{N}(\theta, 1/n)~~~~\textrm{and}~~~~\mathbb{E}((\bar{\tilde{y}}-\bar{y})^2) = \mathbb{E}(\bar{\tilde{y}}^2)-\mathbb{E}(2\bar{\tilde{y}}\bar{y}) + \mathbb{E}(\bar{y}^2) = 2var(\bar{y}) = \frac{2}{n}$$
Thus the entire expectation above gives $n+1$. And thus we obtain
$$elppd = -\frac{n}{2}\log(2\pi)-\frac{n}{2}\log\big(1 + \frac{1}{n}\big) - \frac{n}{2}$$
We will also need the expected value of the log pointwise prediction density for existing data, i.e
$$\mathbb{E}(lppd) = \mathbb{E}(\sum_{i=1}^n\log p_{post}(y_i)) = -\frac{n}{2}\log(2\pi)-\frac{n}{2}\log(1+\frac{1}{n}) - \frac{1}{2}\frac{n(n-1)}{n+1}\mathbb{E}(s_y^2)$$
where the last term is equal to 1. So it is important that elppd is not the same as $\mathbb{E}(lppd)$, one works with future data and the other considers the data that we have observed. I think one difference lies in that the point that we predict over $y_i$ in lppd, is also contained in the corresponding mean $\bar{y}$, and this is not the case for elppd.

The correct number of effective number of parameters is the difference between these two quantities, i.e, [Q10]
$$\mathbb{E}(lppd) - elppd = \frac{n}{n+1}$$
and this quantitity will always be less than 1. 

For AIC and DIC we are interested in the expected out-of-sample log density given a point estimate, this we also calculate
$$\mathbb{E}(\log p(\tilde{y}|\hat{\theta}(y))) = \frac{n}{2}\log(2\pi)-\frac{n}{2}-\frac{1}{2}$$
By taking the expectation of the predicted quantities from $elpd_{AIC}$ and $elpd_{DIC}$ it is possible to show that these are unbiased and thus give the quantiity above. When comparing with the true elppd though there is a difference

## Questions/Answers
* [Q10]: Why is this a good measure of the effective number of parameters? I can understand that this would be a measure of how the bias correction sort of.


----------------------------------------------------------------------
The final part deals with the discussion. They say that their first step in a modelling problem is setting up a simple model, such as a normal model, even if one knows data is discreet etc. After that we can expand on our model and see how it can be improved, these model selection criterias can then be used to see how well the expanded model compares.

It is also not completely clear how large of a value of these information criterias that correspond to a large gain in an improved model, sometimes the gain can be small and sometimes it can be large.

These information criterias and corss-validation try to correct for the fact that we use the data twice, once to fit our models and then to select the model. When these models are used for model selection there can be an issue, if we are choosing between many different models using this as a criteria then it is possible that we obtain s subpar model due to overfitting. They therefore view these procedures as a tool for understanding models and not for selecting between models.

The final comments regard some issues with these criterias:
* AIC: This does not work in situations where we have some strong prior information
* DIC: Does not give good results/nonsensical results when the posterior distribution is not well-summarized by its mean (maybe not good for Neural networks then?)
* WAIC: Relies on data partition which can make it tricky to apply in structured data settings.
* CV: Can be computationally expensive and can also not be well defined in dependent data settings (How can it not be well defined?)
They thus mention that sometimes Bayesian do not always use predictive error comparisons but it has its place due to it being a useful tool to compare models.
Their preferred choice is Cross-validation with WAIC as a fast and computationally efficient alternative.

They would like to see some research into bridging WAIC and CV so that one has the efficiency of the first one and the robustness of the second one.

My main question regarding this article is some intution for why the penalty term takes the shape as it does, we can see that it is unbiased in the limit and such but they look a bit foreign to me in some cases (except AIC).